# Metadata API Introduction

This notebook shows basic use of our metadata API using a Python library PySTAC-Client. It demonstrates how to fetch all collections, fetch a given collection/item, and perform simple searches.

# Connecting to metadata API

We first connect to the metadata API by retrieving the root catalog

To do this, you will need to go to https://dashboard.staging.reefdata.io/ and copy your Authentication token.
This can then be pasted into the password prompt.

In [1]:
from getpass import getpass
import pystac_client

# Metadata STAC API root url
URL = 'https://stac.staging.reefdata.io'

# Go to https://dashboard.staging.reefdata.io/, copy your API key and paste into password box

# Create the client
api = pystac_client.Client.open(
    url="https://stac.staging.reefdata.io/",
    headers={
        'Authorization': f"Bearer {getpass()}"
    },
    ignore_conformance=True
)

api.title

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pystac_client/client.py:177: FutureWarning: The `ignore_conformance` option is deprecated and will be removed in the next major release. Instead use `set_conforms_to` or `add_conforms_to` to control behavior.
  warnings.warn(


'stac-fastapi'

# Fetch all STAC collections

In [2]:
for collection in api.get_collections():
    print(collection)

<CollectionClient id=noaa-crw>
<CollectionClient id=aims-oceanography>
<CollectionClient id=gbrmpa-admin-regions>
<CollectionClient id=abs-census>
<CollectionClient id=qtmr-vessels>
<CollectionClient id=amsa-vessel-tracking>
<CollectionClient id=abs-spatial>
<CollectionClient id=gbr-biogeographic-areas>
<CollectionClient id=gbr-spatial-features>


# Fetch a given collection by ID

In [4]:
collection = api.get_collection('aims-oceanography')
collection

id: aims-oceanography
title: AIMS Oceanography
"description: The Australian Institute of Marine Science (AIMS) has been collecting in situ oceanographic data since 1980 with the deployment of automatic weather observing stations and since 1991 with the sea temperature monitoring program. Both datasets provide hourly data related to sea temperature at a range of depths, atmospheric pressure, air temperature, relative humidity, solar radiation (light as PAR), wind direction and wind speed, spanning the entirety of the Great Barrier Reef."
"providers: AIMS (licensor, producer): AIMS Oceanography"
type: Collection
https://stac-extensions.github.io/projection/v1.1.0/schema.json
https://stac-extensions.github.io/themes/v1.0.0/schema.json
id: aims-sst
"bbox: [96.7556, -34.3725, 159.2372, -2.6651]"
title: AIMS Sea Surface Temperature Monitoring Program
"themes: [{'scheme': 'https://wiki.esipfed.org/ISO_19115-3_Codelists#MD_TopicCategoryCode', 'concepts': [{'id': 'oceans', 'title': 'Oceans'}]}]"


# Fetch all items

The function get_items return iterators, where pystac-client will handle retrieval of additional pages when needed. Note that one request is made for the first ten items, you can make a second request for the next ten.

In [5]:
items = collection.get_items()

# flush stdout so we can see the exact order that things happen
def get_ten_items(items):
    for i, item in enumerate(items):
        print(f"{i}: {item}", flush=True)
        if i == 9:
            return

print('First page', flush=True)
get_ten_items(items)

First page
0: <Item id=aims-sst>


# Fetch a given item


In [6]:
item = collection.get_item('aims-sst')
item

id: aims-sst
"bbox: [96.7556, -34.3725, 159.2372, -2.6651]"
title: AIMS Sea Surface Temperature Monitoring Program
"themes: [{'scheme': 'https://wiki.esipfed.org/ISO_19115-3_Codelists#MD_TopicCategoryCode', 'concepts': [{'id': 'oceans', 'title': 'Oceans'}]}]"
sci:doi: 10.25845/5b4eb0f9bb848
"proj:bbox: [96.7556, -34.3725, 159.2372, -2.6651]"
proj:epsg: 4326
"description: The data provided here are from a number of sea water temperature monitoring programs conducted in tropical and subtropical coral reefs environments around Australia. Data are available from approximately 80 GBR sites, 16 Coral Sea sites, 7 sites in North West Western Australia (WA), 8 Queensland regional ports, 13 sites in the Solitary Islands, 4 sites in PNG and 10 sites in the Cocos (Keeling) Islands. Data are obtained from in-situ data loggers deployed on the reef. Temperature instruments sample water temperatures every 5-10 minutes (typically) and are exchanged and downloaded approximately every 12 months. Temperature loggers on the reef-flat are generally placed just below Lowest Astronomical Tide level. Reef-slope (or where specified as Upper reef-slope) generally refers to depths 5 - 9 m while Deep reef-slope refers to depths of ~20 m. Reefs are under threat from climate change. Elevated sea temperature is a major stress to reefs, capable of causing widespread coral bleaching and mortality. Small excursions of temperature outside normal summer levels can cause stress and bleaching in corals, especially sensitive species. Temperature data from this program are used to correlate with bleaching events and derive local bleaching thresholds. Data are also used in a wide variety of other marine research programs, including fisheries, balast water, turtles, sea birds, seagrass, coral disease, oceanography, process studies, validation of proxy climate records etc."
end_datetime: 2022-05-06T13:56:45Z
"sci:citation: Australian Institute of Marine Science (AIMS). (2017). AIMS Sea Water Temperature Observing System (AIMS Temperature Logger Program). https://doi.org/10.25845/5b4eb0f9bb848, accessed 05-Jun-2023."
"table:columns: [{'name': 'fid', 'type': 'int64'}, {'name': 'deployment_id', 'type': 'int64'}, {'name': 'site', 'type': 'byte_array'}, {'name': 'site_id', 'type': 'int64'}, {'name': 'subsite', 'type': 'byte_array'}, {'name': 'subsite_id', 'type': 'int64'}, {'name': 'from_date', 'type': 'int64'}, {'name': 'thru_date', 'type': 'int64'}, {'name': 'depth', 'type': 'double'}, {'name': 'parameter', 'type': 'byte_array'}, {'name': 'instrument_type', 'type': 'byte_array'}, {'name': 'serial_num', 'type': 'byte_array'}, {'name': 'lat', 'type': 'double'}, {'name': 'lon', 'type': 'double'}, {'name': 'gbrmpa_reef_id', 'type': 'byte_array'}, {'name': 'metadata_uuid', 'type': 'byte_array'}, {'name': 'sites_with_climatology_available', 'type': 'double'}, {'name': 'time', 'type': 'int64'}, {'name': 'cal_val', 'type': 'double'}, {'name': 'qc_val', 'type': 'double'}, {'name': 'qc_flag', 'type': 'double'}, {'name': 'geometry', 'type': 'byte_array'}, {'name': 'hilbert_distance', 'type': 'int64'}]"


# Inspect an item for assets

In [7]:
# Inspect assets
item_assets = item.get_assets()
data_asset = item_assets['data']
if data_asset is not None:
    print(data_asset.to_dict())  

{'href': 's3://rimrep-data-public/091-aims-sst/test-50-64-spatialpart/', 'type': 'application/x-parquet', 'title': 'AIMS Sea Water Temperature Observing System (AIMS Temperature Logger Program)', 'description': 'S3 address of the AIMS Sea Water Temperature Observing System (AIMS Temperature Logger Program) in GeoParquet format', 'roles': ['data']}


# Inspect an item for link to data API

In [8]:
# Inspect link to data API
link = item.get_single_link(rel="describedby")
if link is not None:
    print(link.to_dict())

{'rel': 'describedby', 'href': 'https://pygeoapi.staging.reefdata.io/collections/aims-sst', 'title': 'Link to data API'}


# Search for items by spatial and temporal extent

In [9]:
geom = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          162,
          -33
        ],
        [
          162,
          -3
        ],
        [
          136,
          -3
        ],
        [
          136,
          -33
        ],
        [
          162,
          -33
        ]
      ]
    ]
}

results = api.search(
    max_items = 15,
    limit = 5,
    intersects = geom,
    datetime = "2000-01-01/2023-03-07",
)

for item in results.items():
    print(item.id)

abs-lgas-2022
amsa-ais
abs-postal-areas-2021
abs-lgas-2021
gbrmpa-benthic-map
qtmr-recreational-boat-facilities
dcceew-coral-sea-boundary
gbrmpa-gbr-river-basins
qtmr-vessel-registrations
abs-regional-lga-2021
gbrmpa-traditional-use-areas
gbrmpa-special-management-areas
gbrmpa-management-areas
gbrmpa-marine-park-limits
gbrmpa-whale-protection-area


# Search for items using query

In [11]:
# Search for items using a query to filter by keywords
# Currently "contains" operator is not supported, so you can only "eq" operator find exact matches

results = api.search(
    max_items = 15,
    limit = 5,
    query={"themes": {"eq": [{'scheme': 'https://wiki.esipfed.org/ISO_19115-3_Codelists#MD_TopicCategoryCode',
                              'concepts': [{'id': 'oceans', 'title': 'Oceans'}]}]}}
)

for item in results.items():
    print(item.id)

dcceew-coral-sea-boundary
aims-sst
gbrmpa-complete-gbr-features
